# Predicting heart disease using machine learning

This notebook looks into using various Python-based machine learning data science libraries in attempt to build a machine learning model capable of predicting whether or not someone has heart disease based on their medical attributes.

1. Problem definition
2. Data 
3. Evaluation
4. Features
5. Modeling
6. Experimentation

## 1. Problem Definition

> Given clinical parameters about a patient, can we predict whether or not they have heart disease?

## 2. Data

The original data came from the Cleavland data from the UCI Machine Learning Repository. 
https://archive.ics.uci.edu/ml/datasets/Heart+Disease

There is also a version of it on Kaggle.
https://www.kaggle.com/datasets/cherngs/heart-disease-cleveland-uci

## 3. Evaluation
